# README

This notebook pretrains the ResNet Encoder Weights on the HealNet Classifications task

# Pytorch HealNet
Run the cells in this section to load the components of HealNet and train.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import numpy as np
import time
import torch
import copy
from PIL import Image
from tqdm.notebook import trange, tqdm
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, ImageReadMode
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Mouse Dataset

In [ ]:
class Wound_Dataset(Dataset):

    def __init__(self, root_dir, transform=None):
        """
        Wound dataset. Returns tensorized pairs of images (Left, Right) with
        corresponding class 0: negative, 1: positive
        Args:
            root_dir: path to a cropped mouse image dataset.
        """
        self.root_dir = root_dir
        self.transform = transform 
        self.samples, self.targets = self.make_dataset(self.root_dir)

    def make_dataset(self, root_dir):
        """creates a list of tuples containing paths to data in root data directory
        plus the labels for the dataset.

        Args:
            root_dir (string): root directory with subdirectory containing .png 
            files of mouse wound images
            class_to_idx: dictionary of classes and numerics.
        Returns:
            instances (tuple): list of tuples with path to image files and 
                list targets.
        """
        # Create Dictionary Containing all the images for the same mouse/wound
        wounds = {}
        for f in os.listdir(root_dir):

            tmp = f.split("_")
            day = tmp[0]
            mouse_wound = tmp[1].split(".")[0]
            if mouse_wound in wounds.keys():
                wounds[mouse_wound].append((int(day.split(" ")[1]),f))
            else:
                wounds[mouse_wound] = [(int(day.split(" ")[1]),f)]

        # Populate lists of positive and negative pairs 
        neg = []
        pos = []

        for wound in wounds.keys():
            tmp = sorted(wounds[wound], key=lambda x: x[0])
  
            # Generate list of positives:
            for i in range(len(tmp)):
                if i != len(tmp)-1:
                    for j in range(i+1,len(tmp)):
                        pos.append((root_dir+"/"+tmp[i][1], root_dir+"/"+tmp[j][1]))

            # Generate list of negatives:
            for i in range(len(tmp)-1,-1, -1):
                if i != 0:
                    for j in range(i-1,-1,-1):
                        neg.append((root_dir+"/"+tmp[i][1], root_dir+"/"+tmp[j][1]))

        # Generate Labels
        neg_labels = [0]*len(neg)
        pos_labels = [1]*len(neg)

        # Combine positive and negatives lists
        neg.extend(pos)

        # Combine positive and negatives labels
        neg_labels.extend(pos_labels)

        """ Don't Need in Pytorch; Dataloader can shuffle
        # Shuffle negs and labels with same order
        rng = np.random.RandomState(seed=42)
        temp = list(zip(neg, neg_labels))
        rng.shuffle(temp)

        #Unzip shuff
        res1, res2 = zip(*temp)
        # res1 and res2 come out as tuples, and so must be converted to lists.
        instances, targets = list(res1), list(res2)
        """ 
        instances = neg
        targets = neg_labels
        return instances, targets

    def pil_loader(self, path):
        # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

    def __getitem__(self, index: int):
            """
            Returns tensor data and label.
            Args:
                index (int): Index
            Returns:
                tuple: (sample, target) where sample is a tensor of input data
                        target is class of sample.
            """
            # Get images and targets, drop alpha
            left_img = self.pil_loader(self.samples[index][0])
            right_img = self.pil_loader(self.samples[index][1])
            target = self.targets[index]


            if self.transform:
                left_img = self.transform(left_img)
                right_img = self.transform(right_img)

            # Concat and return sample, target
            return torch.stack((left_img, right_img)) , target

    def __len__(self):
        return len(self.samples)

## Data loader

In [ ]:
def wound_collate(batch):
    """
    custom collate function for loading a batch of 2-image tensors and target
    
    Args:
        Batch: batch of data called by dataloader
    
    Returns:
        tuple: (sample, target) sample is 2x3xHxW tensor;target is class of sample
    """
    # Form a tensor of first tuple items in batch
    data = torch.stack([item[0] for item in batch])  
    # Tensorize targets
    target = [item[1] for item in batch]
    target = torch.LongTensor(target)
    # Return tuple of data & targets
    return [data, target]

In [ ]:
def wound_dataloader(dataset, batch_size: int, num_workers = 0, shuffle = False, pin_memory = False):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=wound_collate,
        num_workers=num_workers,
        shuffle = shuffle,
        pin_memory = pin_memory)

## Healnet Model For Forward/Backwards Classification

In [ ]:
class Encoder(nn.Module):
    """
    Encoder model Pytorch. 
    """   
    def __init__(self):
        # Initialize self._modules as OrderedDict
        super(Encoder, self).__init__() 
        # Initialize densenet121
        self.embed_model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=False)
        # Remove Classifying layer
        self.embed_model = nn.Sequential(*list(self.embed_model.children())[:-1])
        # 7x7 average pool layer
        self.avg = nn.AvgPool2d(kernel_size=7, stride=1)
        # Left image connected layers
        self.fc_16 = nn.Linear(1024, 16)


    def forward(self, x):
        # Embed Left
        u1 = self.embed_model(x)
        u1 = self.avg(u1)
        u1 = u1.view(-1,1024)
        u1 = self.fc_16(u1)
        #u1 = torch.relu(u1)
        return u1

    def load_embed_wts(self, device):
        """
        load pretrained model weights, use only when transfer learning from ImageNET data
        """
        # Initialize densenet121
        self.embed_model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True).to(device)

        # Remove Classifying layer
        self.embed_model = nn.Sequential(*list(self.embed_model.children())[:-1])

class ResnetEncoder(nn.Module):
    """
    ResnetEncoder model Pytorch. 
    """   
    def __init__(self):
        # Initialize self._modules as OrderedDict
        super(ResnetEncoder, self).__init__() 
        # Initialize resnet18
        self.embed_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
        # Classification Layer
        num_ftrs = self.embed_model.fc.in_features
        self.embed_model.fc = nn.Linear(num_ftrs, 16)
        self.fc = self.embed_model.fc


    def forward(self, x):
        # Embed Left
        return self.embed_model(x)

    def load_embed_wts(self, device):
        """
        load pretrained model weights, use only when transfer learning from ImageNET data
        """
        # Initialize resnet18
        self.embed_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True).to(device)

        # Classifying layer
        self.embed_model.fc = self.fc.to(device)

class HealNet(nn.Module):
    """
    HealNet model Pytorch. 
    """
    def __init__(self):
        # Initialize self._modules as OrderedDict
        super(HealNet, self).__init__() 
        self.encoder = ResnetEncoder()
        # Classification Layer
        self.fc_classify = nn.Linear(32, 2)

    def load_encoder_wts(self,device):
        """
        load pretrained model weights, use only when transfer learning from ImageNET data
        """
        # Initialize densenet121
        self.encoder.load_embed_wts(device)


    def forward(self, x):
        """
        forward call to Healnet
        Args:
            x: a tensorized tuple of mouse images of size BSx2x3x224x224
                in the form Batch size by (left image, right image).
        Return:
            preds: logits for BCE.
        """

        # Embed Left
        u1 = self.encoder(x[:,0,:,:,:])


        # Embed Reft
        u2 = self.encoder(x[:,1,:,:,:])

        #Return logits
        return self.fc_classify(torch.cat((u1,u2),1))

## Training Loop

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase],desc='batches', leave = False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

## Train Model

### Augmentation

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation

input_size = 224 # Muse be 224 (3x224x224) for Densenet121

# Standard Pytorch image transforms (source:https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

### Train

In [ ]:
!ls gdrive/ 

MyDrive  Shareddrives


In [ ]:
# Hyper Parameters:
data_path = '/content/gdrive/MyDrive/Explainable_Wound_Classification/Split_images' #path to Wound_images
learning_rate = 0.005
weight_decay = 0.0
batch_size = 16
num_epochs = 50

# Generate Datasets & Loaders
data_loaders_dict = {phase: wound_dataloader(Wound_Dataset(data_path+"/"+phase, transform=data_transforms[phase]), 
                                             batch_size, num_workers = 0, shuffle = True, pin_memory = True) for phase in ['train', 'val']}

# Create Model
healnet = HealNet()
healnet.to(device)
healnet.load_encoder_wts(device)

# Specify optimizer and loss function
optimizer = torch.optim.Adam(healnet.parameters(), lr=learning_rate, weight_decay=weight_decay)
lossfun = nn.CrossEntropyLoss()

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
healnet, hist = train_model(healnet, data_loaders_dict, criterion=lossfun, optimizer = optimizer, num_epochs=num_epochs)

Epoch 0/49
----------


batches:   0%|          | 0/179 [00:00<?, ?it/s]

train Loss: 0.5612 Acc: 0.7316


batches:   0%|          | 0/30 [00:00<?, ?it/s]

val Loss: 0.8417 Acc: 0.7000

Epoch 1/49
----------


...

val Loss: 0.2644 Acc: 0.8875

Epoch 49/49
----------


batches:   0%|          | 0/179 [00:00<?, ?it/s]

train Loss: 0.2098 Acc: 0.9165


batches:   0%|          | 0/30 [00:00<?, ?it/s]

val Loss: 0.2567 Acc: 0.8875

Training complete in 42m 4s
Best val Acc: 0.918750


In [ ]:
# Save Healnet Model
save_as = "/content/gdrive/MyDrive/Explainable_Wound_Classification/resnet_healnet_weights.tar" # path and name of trained healNet weights
torch.save(healnet.state_dict(), save_as)

# Save Healnet.encoder weights 
save_as_encoder = "/content/gdrive/MyDrive/Explainable_Wound_Classification/resnet_healnet_encoder_weights.tar" # path and name of trained encoder weights
encoder = healnet.encoder
torch.save(encoder.state_dict(), save_as_encoder)

## Get Stats On data

In [ ]:
def model_val(model, dataloaders, loss):
    """ train a model with given params
    Args:
        model: model, extends torch.nn
        dataloaders: dataloader 
        loss: loss fn
        
    
    Return:
        Tuple: model with trained weights and validation training statistics(epoch loss, accuracy)
    """
    all_preds = []
    all_labels = []
    
    #isntantiate validation history, base model waits and loss
    val_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    model.eval()   # Set model to evaluate mode 

    running_loss = 0.0
    running_corrects = 0  

    for batch in tqdm(dataloaders,desc='batch', leave = False):
        #send inputs and labels to device
        inputs = batch[0].to(device)
        labels = batch[1].to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):

            # Get model outputs and calculate loss for val
            preds = model(inputs)
            loss = lossfun(preds, labels) 

            #get predictions
            _, preds = torch.max(preds, 1)

            #running statistics
            all_preds.append(preds)
            all_labels.append(labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    #update epoch loss and acc
    epoch_acc = running_corrects.double() / len(dataloaders.dataset)

    #track validation loss and acc
    tqdm.write('Dataset Classification Accuracy: {:.4f}'.format(epoch_acc))

    #return all_preds, all_labels

In [ ]:
# Load Model
healnet = HealNet()
healnet.to(device)
path_to_healnet = "/content/gdrive/MyDrive/Explainable_Wound_Classification/resnet_healnet_weights.tar" # Path to healnet weights
healnet.load_state_dict(torch.load(path_to_healnet))

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip


<All keys matched successfully>

In [ ]:
# Generate Datasets & Loaders
data_path = '/content/gdrive/MyDrive/Explainable_Wound_Classification/Split_images' # Path to wound images
batch_size = 16
lossfun = nn.CrossEntropyLoss()

# Recreate Dataloaders Dictionary with only val transforms applied
data_loaders_dict = {phase: wound_dataloader(Wound_Dataset(data_path+"/"+phase, transform=data_transforms['val']), 
                                             batch_size, num_workers = 0, shuffle = False, pin_memory = True) for phase in ['train', 'val', 'test']}


## Model Classification Results

In [ ]:
# Get Training Accuracy
#model_val(healnet,data_loaders_dict['train'],  loss = lossfun)

# Get Val Accuracy
model_val(healnet,data_loaders_dict['val'],  loss = lossfun)

# Get Test Accuracy
model_val(healnet,data_loaders_dict['test'],  loss = lossfun)

batch:   0%|          | 0/30 [00:00<?, ?it/s]

Dataset Classification Accuracy: 0.9187


batch:   0%|          | 0/30 [00:00<?, ?it/s]

Dataset Classification Accuracy: 0.8708
